In [33]:
import time
import torch
import sbibm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import io

from pydmd import DMDc
np.set_printoptions(suppress=True)

In [34]:
task_name = "two_moons"
device = "cpu"
task = sbibm.get_task(task_name)
prior = task.get_prior_dist()
simulator = task.get_simulator()

In [35]:
cached_fn = os.path.join("trained", f"{task_name}.nf")
with open(cached_fn, "rb") as f:
    encoder = pickle.load(f)
encoder.to(device)

Flow(
  (_transform): CompositeTransform(
    (_transforms): ModuleList(
      (0): PiecewiseRationalQuadraticCouplingTransform(
        (transform_net): ResidualNet(
          (initial_layer): Linear(in_features=3, out_features=50, bias=True)
          (blocks): ModuleList(
            (0-1): 2 x ResidualBlock(
              (context_layer): Linear(in_features=2, out_features=50, bias=True)
              (linear_layers): ModuleList(
                (0-1): 2 x Linear(in_features=50, out_features=50, bias=True)
              )
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (final_layer): Linear(in_features=50, out_features=29, bias=True)
        )
      )
      (1): LULinear()
      (2): PiecewiseRationalQuadraticCouplingTransform(
        (transform_net): ResidualNet(
          (initial_layer): Linear(in_features=3, out_features=50, bias=True)
          (blocks): ModuleList(
            (0-1): 2 x ResidualBlock(
              (context_layer):

In [43]:
def create_system_snapshots(A, B):
    n = B.shape[0]
    m = 25 # number of snapshots

    x0 = np.array([0.25] * n)
    u = np.random.rand(n, m - 1) - .5
    print(B.shape)
    print(u[:, 0].shape)
    
    snapshots = [x0]
    for i in range(m - 1):
        snapshots.append(A.dot(snapshots[i]) + B.dot(u[:, i]))
    snapshots = np.array(snapshots).T
    return A, B, {'snapshots': snapshots, 'u': u, 'B': B, 'A': A}

In [46]:
test_sim = 1
test_theta = prior.sample((test_sim,))
test_x = simulator(test_theta)
A = np.hstack([test_theta, test_theta]).reshape((2, 2))
B = np.hstack([test_theta, test_theta]).reshape((2, 2))

In [47]:
A, B, system = create_system_snapshots(A, B)
dmdc = DMDc(svd_rank=-1, opt=True)
dmdc.fit(system['snapshots'], system['u'])
A_hat, B_hat, _ = dmdc.reconstructed_data()

(2, 2)
(2,)


/home/yppatel/anaconda3/envs/chig/lib/python3.8/site-packages/pydmd/snapshots.py:72: UserWarning: Input data condition number 1.1234838604139866e+16. Consider preprocessing data, passing in augmented data
matrix, or regularization methods.
  warnings.warn(


In [50]:
print(f"A = {A}")
print(f"A_hat = {np.real(A_hat)}")

A = [[0.80859303 0.89611864]
 [0.80859303 0.89611864]]
A_hat = [[0.85235584 0.85235584]
 [0.85235584 0.85235584]]


In [51]:
print(f"B = {B}")
print(f"B_hat = {np.real(B_hat)}")

B = [[0.80859303 0.89611864]
 [0.80859303 0.89611864]]
B_hat = [[0.80859303 0.89611864]
 [0.80859303 0.89611864]]


In [49]:
# done: 1. generate C
# done: 2. learn generative model for C^ (seems like it'll be identical since we can just consider C to be a reshaped vector)
# done: 3. simulate trajectories w/ C/C_hat (sort of separate, but can do concurrently)
# 4. recover C^ from trajetories 
# 5. learn optimal controller for C_hat